In [46]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn.utils.rnn import pad_sequence

In [36]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Mental_Health_FAQ.csv')

# Separate questions and answers
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

In [37]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [49]:
# Add a new pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the questions and answers
input_ids = tokenizer(questions, return_tensors='pt', padding=True, truncation=True)['input_ids']
labels = tokenizer(answers, return_tensors='pt', padding=True, truncation=True)['input_ids']

# Padding sequences to a fixed length
max_length = max(input_ids.size(1), labels.size(1))
input_ids_padded = pad_sequence([torch.cat((seq, torch.full((max_length - seq.size(0),), tokenizer.pad_token_id))) for seq in input_ids], batch_first=True)
labels_padded = pad_sequence([torch.cat((seq, torch.full((max_length - seq.size(0),), tokenizer.pad_token_id))) for seq in labels], batch_first=True)


In [50]:
from torch.utils.data import Dataset, DataLoader

# Create dataset and dataloader
class MentalHealthDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'labels': self.labels[idx]}

dataset = MentalHealthDataset(input_ids_padded, labels_padded)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Fine-tune the GPT-2 model
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in dataloader:
        inputs = batch['input_ids']
        labels = batch['labels']

        # Mask out padding tokens
        attention_mask = (inputs != tokenizer.pad_token_id).float()

        optimizer.zero_grad()

        try:
            outputs = model(inputs, labels=labels, attention_mask=attention_mask)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        except Exception as e:
            print(f"Error in epoch {epoch}, batch {batch}: {e}")

Error in epoch 0, batch {'input_ids': tensor([[ 8496,   460,   314,  ..., 50257, 50257, 50257],
        [ 8496,   460,   314,  ..., 50257, 50257, 50257]]), 'labels': tensor([[24704,    12, 16794,  ..., 50257, 50257, 50257],
        [ 7085,   661,  1064,  ..., 50257, 50257, 50257]])}: index out of range in self
Error in epoch 0, batch {'input_ids': tensor([[ 8496,   460,   314,  ..., 50257, 50257, 50257],
        [ 2061,   857,   340,  ..., 50257, 50257, 50257]]), 'labels': tensor([[40341,  6982,   286,  ..., 50257, 50257, 50257],
        [   44,  2470, 24024,  ..., 50257, 50257, 50257]])}: index out of range in self
Error in epoch 0, batch {'input_ids': tensor([[ 2061,   338,   262,  ..., 50257, 50257, 50257],
        [ 2061,   318,   262,  ..., 50257, 50257, 50257]]), 'labels': tensor([[26699,  1108,   318,  ..., 50257, 50257, 50257],
        [ 1858,   389,   867,  ..., 50257, 50257, 50257]])}: index out of range in self
Error in epoch 0, batch {'input_ids': tensor([[ 8241,   857,  51

In [54]:
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model\\tokenizer_config.json',
 'fine_tuned_model\\special_tokens_map.json',
 'fine_tuned_model\\vocab.json',
 'fine_tuned_model\\merges.txt',
 'fine_tuned_model\\added_tokens.json')

In [64]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_model')

# Function to generate responses
def generate_response(user_input):
    input_ids = tokenizer.encode(user_input, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)
    output = model.generate(input_ids, max_length=200, attention_mask=attention_mask, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, do_sample=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage
user_input = "How can I cope with stress?"
response = generate_response(user_input)
print(response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I cope with stress?

The answer to this question depends on your level of stress. The more stress you experience, the more likely it is that you will be stressed. If you are chronically stressed, you may not be able to focus on the task at hand. You may feel as if you have lost control of your life. This can cause you to feel like you're in a constant state of anxiety, which can lead to depression. It's important to remember that stress is not the only thing that can affect your health. Stress can also affect the way you think, feel, and think about yourself. For example, stress can make it difficult for people to relate to one another. People who are stressed tend to be more pessimistic about their future. They are less likely to believe that things will go their way or that they will make the right decisions. In other words, people who feel stressed may find it hard to connect with their friends and loved ones. There are many


In [66]:
user_input = "What is Mental Health?"
response = generate_response(user_input)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is Mental Health?

Mental health is defined as having a mental disorder that is severe enough to cause a person to become depressed, anxious, or anxious in the future. It is a condition that can lead to serious health problems, such as heart disease, stroke, diabetes, and cancer. Mental health can also be associated with a number of other mental health conditions, including depression, anxiety, obsessive-compulsive disorder (OCD), bipolar disorder, schizophrenia, post-traumatic stress disorder and borderline personality disorder. The Diagnostic and Statistical Manual of Mental Disorders, Fifth Edition (DSM-5) defines mental illness as "a mental condition in which an individual is characterized by a persistent or persistent lack of self-control or control over his or her behavior, feelings, thoughts, actions and/or behavior." Mental illness is also sometimes referred to as a "disorder of the mind," a term that has been used to describe people who are unable to control their thought

In [67]:
user_input = "Can people with mental illness recover?"
response = generate_response(user_input)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Can people with mental illness recover?

The answer to this question depends on a number of factors, including whether the person has been diagnosed with schizophrenia or bipolar disorder, and whether they have been treated with antipsychotic medication. In general, people who are at high risk for relapse may be at higher risk of relapse than those who do not have a history of psychiatric illness. However, there is no evidence to support the idea that schizophrenia and bipolar disorders are linked, or that they are related, to each other. It is important to note that there are many factors that can affect a person's ability to recover from a relapse. For example, it is possible that a patient with a psychiatric disorder may not be able to return to work for a long period of time because of an underlying illness, such as depression or anxiety, which may affect the ability of the patient to function effectively in the workplace. A person with an illness that is not related to their illne

In [65]:
# Example usage
user_input = "How can I find a mental health professional for myself or my child?"
response = generate_response(user_input)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I find a mental health professional for myself or my child?

There are many ways to get help for depression, anxiety, and other mental illnesses. For more information, see our Mental Health Resources page.
